# 第七章 文本扩展

<div class="toc">
    <ul class="toc-item">
        <li><span><a href="#一引言" data-toc-modified-id="一、引言">一、引言</a></span></li>
        <li>
            <span><a href="#二定制客户邮件" data-toc-modified-id="二、定制客户邮件">二、定制客户邮件</a></span>
        </li>
        <li><span><a href="#三引入温度系数" data-toc-modified-id="三、引入温度系数">三、引入温度系数</a></span>
        </li>
    </ul>
</div>

## 一、引言

扩展是将短文本（例如一组说明或主题列表）输入到大型语言模型中，让模型生成更长的文本（例如基于某个主题的电子邮件或论文）。这种应用是一把<span style="color:red">双刃剑</span>，好处例如将大型语言模型用作头脑风暴的伙伴；但也存在问题，例如某人可能会使用它来生成大量垃圾邮件。因此，当你使用大型语言模型的这些功能时，请仅以**负责任** (responsible) 和**有益于人们** (helps people) 的方式使用它们。

在本章中，你将学会如何基于 OpenAI API 生成*针对每位客户评价优化*的客服电子邮件。我们还将利用模型的另一个输入参数称为温度，这种参数<span style="color:red">允许您在模型响应中变化探索的程度和多样性</span>。

同以上几章，你需要类似的代码来配置一个可以使用 OpenAI API 的环境

In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv

# 读取本地/项目的环境变量。

# find_dotenv()寻找并定位.env文件的路径
# load_dotenv()读取该.env文件，并将其中的环境变量加载到当前的运行环境中  
# 如果你设置的是全局的环境变量，这行代码则没有任何作用。
_ = load_dotenv(find_dotenv())

# 获取环境变量 OPENAI_API_KEY
openai_api_key = os.getenv("OPENAI_API_KEY")

In [4]:
# 初始化客户端
client = OpenAI(api_key=openai_api_key)

# 一个封装 OpenAI 接口的函数，参数为 Prompt，返回对应结果
def get_completion(prompt, model="gpt-4o-mini"):
    '''
    prompt: 对应的提示词
    model: 调用的模型，默认为gpt-4o-mini
    '''
    # 调用API
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0,  # 模型输出的温度系数，控制输出的随机程度
    )
    # 返回结果
    return response.choices[0].message.content

## 二、定制客户邮件

我们将根据客户评价和情感，<span style="color:red">针对性</span>写自动回复邮件。因此，我们将给定客户评价和情感，使用 LLM 针对性生成响应，即根据客户评价和评论情感生成定制电子邮件。

我们首先给出一个示例，包括一个评论及对应的情感。

In [3]:
# 我们可以在推理那章学习到如何对一个评论判断其情感倾向
sentiment = "negative"

# 一个产品的评价
review = f"""
他们在11月份的季节性销售期间以约49美元的价格出售17件套装，折扣约为一半。\
但由于某些原因（可能是价格欺诈），到了12月第二周，同样的套装价格全都涨到了70美元到89美元不等。\
11件套装的价格也上涨了大约10美元左右。\
虽然外观看起来还可以，但基座上锁定刀片的部分看起来不如几年前的早期版本那么好。\
不过我打算非常温柔地使用它，例如，\
我会先在搅拌机中将像豆子、冰、米饭等硬物研磨，然后再制成所需的份量，\
切换到打蛋器制作更细的面粉，或者在制作冰沙时先使用交叉切割刀片，然后使用平面刀片制作更细/不粘的效果。\
制作冰沙时，特别提示：\
将水果和蔬菜切碎并冷冻（如果使用菠菜，则轻轻煮软菠菜，然后冷冻直到使用；\
如果制作果酱，则使用小到中号的食品处理器），这样可以避免在制作冰沙时添加太多冰块。\
大约一年后，电机发出奇怪的噪音，我打电话给客服，但保修已经过期了，所以我不得不再买一个。\
总的来说，这些产品的总体质量已经下降，因此它们依靠品牌认可和消费者忠诚度来维持销售。\
货物在两天内到达。
"""

我们已经使用推断课程中所学方法提取了情感，这是一个关于搅拌机的客户评价，现在我们将根据情感定制回复。

以下述 Prompt 为例：假设你是一个客户服务 AI 助手，你的任务是为客户发送电子邮件回复，根据通过三个反引号分隔的客户电子邮件，生成一封回复以感谢客户的评价。

In [5]:
prompt = f"""
你是一位客户服务的AI助手。
你的任务是给一位重要客户发送邮件回复。请根据客户通过三个反引号分隔的评价，生成回复以感谢客户的评价。

以下是生成的回复要求：
1.使用评价中的具体细节。
2.参考客户评论的情感倾向进行书写。
3.用简明而专业的语气写信。
4.作为“AI客户代理”签署电子邮件。
5.不要生成除回复邮件之外的任何内容。

客户评论：
```{review}```
评论情感：{sentiment}
"""
response = get_completion(prompt)
print(response)

尊敬的客户，

感谢您抽出时间分享您的反馈。我们非常重视您的意见，并对您在11月份季节性销售期间的购物体验感到遗憾。我们理解您对价格波动的担忧，尤其是同样的套装在12月的价格上涨，这确实可能让人感到困惑。

关于您提到的产品质量问题，我们深感抱歉。我们一直致力于提供高质量的产品，您的反馈将帮助我们进一步改进。我们也很高兴您分享了使用技巧，这对其他客户来说非常有帮助。

对于您在使用过程中遇到的电机噪音问题，我们建议您在未来的购买中考虑延长保修服务，以便在出现问题时能够得到及时的支持。

再次感谢您的反馈，我们会继续努力提升产品和服务质量。如有任何其他问题或建议，请随时与我们联系。

祝好，

AI客户代理


## 三、引入温度系数

接下来，我们将使用语言模型的一个称为<span style="color:red">“温度” (Temperature) </span>的参数，它将允许我们改变模型响应的多样性。您可以将温度视为<span style="color:red">模型探索或随机性的程度</span>。

例如，在一个特定的短语中，“我的最爱食品”最有可能的下一个词是“比萨”，其次最有可能的是“寿司”和“塔可”。因此，在温度为零时，模型将总是选择最有可能的下一个词，而在较高的温度下，它还将选择其中一个不太可能的词，在更高的温度下，它甚至可能选择塔可，而这种可能性仅为五分之一。您可以想象，随着模型继续生成更多单词的最终响应，“我的最爱食品是比萨”将会与第一个响应“我的最爱食品是塔可”产生差异。随着模型的继续，这两个响应也将变得越来越不同。

一般来说，在构建需要<span style="color:red">可预测响应</span>的应用程序时，我建议**设置温度为零**。在所有课程中，我们一直设置温度为零，如果您正在尝试构建一个可靠和可预测的系统，我认为您应该选择这个温度。如果您尝试<span style="color:red">以更具创意的方式</span>使用模型，可能需要更广泛地输出不同的结果，那么您可能需要使用更高的温度。

同一段来信，我们提醒模型使用用户来信中的详细信息，并设置温度：

In [8]:
prompt = f"""
你是一位客户服务的AI助手。
你的任务是给一位重要客户发送邮件回复。请根据客户通过三个反引号分隔的评价，生成回复以感谢客户的评价。

以下是生成的回复要求：
1.使用评价中的具体细节。
2.参考客户评论的情感倾向进行书写。
3.用简明而专业的语气写信。
4.作为“AI客户代理”签署电子邮件。
5.不要生成除回复邮件之外的任何内容。

客户评论：
```{review}```
评论情感：{sentiment}
"""
response = get_completion(prompt, temperature=0.7)
print(response)

尊敬的客户：  

感谢您抽出时间分享详细的反馈。我们非常重视您的体验，并诚挚为产品价格变动及质量问题带来的不便道歉。  

我们理解您对12月价格调整的担忧，您的反馈已记录并将提交至相关部门审查。关于刀片基座设计的变化，我们会将您的比较意见转达至产品团队，以推动优化。同时，对于电机在一年后出现异常噪音的情况，我们正在评估延长保修期的可能性，以更好地支持客户。  

您分享的使用技巧（如预先处理食材的分步方法）极具价值，我们已将此内容加入客户指南优化提案中。若未来有任何产品使用疑问，欢迎随时联系24小时在线支持团队。  

再次感谢您指出改进方向，您的洞察力将持续帮助我们提升服务标准。  

此致  
敬礼  
AI客户代理


<span style="color:#00ff00">
通常来说，即使设置温度为0，相同的prompt也不一定产生完全一样的输出。这与模型的实现细节与上下文等相关。但是，我们可以期待，温度为0时生成的结果更加稳定可靠。
</span>

在温度为零时，每次执行相同的 Prompt ，您获得的回复理应相同。而使用温度为 0.7 时，则每次都会获得不同的输出。

所以，您可以看到它与我们之前收到的电子邮件不同。再次执行将再次获得不同的电子邮件。

因此，我建议您自己尝试温度，以查看输出如何变化。总之，在更高的温度下，模型的输出更加随机。您几乎可以将其视为在更高的温度下，助手**更易分心**，但也许**更有创造力**。